# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
            
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')
song_files = [i for i in song_files if 'ipynb_checkpoints' not in i]

In [5]:
song_files

['/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/workspace/data/song_data/A/A/A/TRAAARJ128F9320760.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAPK128E0786D96.json',
 '/home/workspace/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMO128F1481E7F.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAVO128F93133D4.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAEF128F4273421.json',
 '/home/workspace/data/song_data/A/A/A/TRAAAFD128F92F423A.json',
 '/home/workspace/data/song_data/A/A/A/TRAAADZ128F9348C2E.json',
 '/home/workspace/data/song_data/A/A/B/TRAABNV128F425CEE1.json',
 '/home/workspace/data/song_data/A/A/B/TRAABRB128F9306DD5.json',
 '/home/workspace/data/song_data/A/A/B/TRAABLR128F423B7E3.json',
 '/home/workspace/data/song_data/A/A/B/TRAABDL12903CAABBA.json',
 '/home/workspace/data/so

In [7]:
# filepath = song_files[10]
df_song = pd.read_json(song_files[0], lines=True)
# df_allsong = pd.DataFrame()
# for i in song_files:
#     df_song = pd.read_json(i, lines=True)
#     print('appending file {}'.format(i))
#     df_allsong = df_allsong.append(df_song, ignore_index=True)
#     print (df_allsong.count())
    

In [8]:
# df = pd.read_json(filepath, lines=True)
# df
# df_allsong.head(10)
df_song

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [21]:
# song_data = df_allsong[['artist_id', 'song_id', 'title', 'year', 'duration']].drop_duplicates().values
song_data = df_song[['artist_id', 'song_id', 'title', 'year', 'duration']].drop_duplicates().values
# song_data = song_data[0].tolist()
song_data = tuple(song_data[0])

song_data

('ARD7TVE1187B99BFB1', 'SOMZWCG12A8C13C480', "I Didn't Mean To", 0, 218.93179)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [20]:
# from sql_queries import song_table_insert
# song_table_insert = ("INSERT INTO songs (artist_id, song_id, title, year, duration )\
#                     VALUES (%s, %s, %s, %s, %s)")
# for song in song_data:
#     try:
#         cur.execute(song_table_insert, song)
#     except psycopg2.Error as e:
#         print(e)
# #     conn.commit()



cur.execute(song_table_insert, song_data)

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df_allsong[['artist_id', 'artist_name', 'artist_location',
                          'artist_latitude', 'artist_longitude' ]].drop_duplicates().values
artist_data = [tuple(i) for i in artist_data]
artist_data

[('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan),
 ('ARNTLGG11E2835DDB9', 'Clp', '', nan, nan),
 ('AR8ZCNI1187B9A069B', 'Planet P Project', '', nan, nan),
 ('AR10USD1187B99F3F1',
  'Tweeterfriendly Music',
  'Burlington, Ontario, Canada',
  nan,
  nan),
 ('ARMJAGH1187FB546F3', 'The Box Tops', 'Memphis, TN', 35.14968, -90.04892),
 ('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', '', nan, nan),
 ('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan),
 ('ARGSJW91187B9B1D6B', 'JennyAnyKind', 'North Carolina', 35.21962, -80.01955),
 ('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', nan, nan),
 ('ARXR32B1187FB57099', 'Gob', '', nan, nan),
 ('ARKRRTF1187B9984DA', 'Sonora Santanera', '', nan, nan),
 ('ARIG6O41187B988BDD',
  'Richard Souther',
  'United States',
  37.16793,
  -95.84502),
 ('AR1ZHYZ1187FB3C717', 'Faiz Ali Faiz', '', nan, nan),
 ('ARD842G1187B997376',
  'Blue Rodeo',
  'Toronto, Ontario, Canada',
  43.64856,
  -79.38533),
 ('ARL7K851187B99ACD2', 'Andy Andy', '', nan

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
artist_table_insert = ("INSERT INTO artists (artist_id, name, location,latitude, longitude)\
                    VALUES (%s, %s, %s, %s, %s)")
for artist in artist_data:
    try:
        cur.execute(artist_table_insert, artist)
    except psycopg2.Error as e:
        print(e)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the __first__ log file in this list
- Read the log file and view the data

In [4]:
log_files = get_files('data/log_data')
log_files = [i for i in log_files if 'ipynb_checkpoints' not in i]

In [5]:
log_files

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

In [6]:
filepath = log_files[0]

In [7]:
df = pd.read_json(filepath, lines=True)
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [8]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
df = df[df['page']=='NextSong']
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


In [10]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()

0   2018-11-30 00:22:07.796
1   2018-11-30 01:08:41.796
2   2018-11-30 01:12:48.796
3   2018-11-30 01:17:05.796
4   2018-11-30 01:20:56.796
Name: ts, dtype: datetime64[ns]

In [11]:
# time_data = ()
# column_labels = ()
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df['start_time'] = pd.to_datetime(df['ts'], unit='ms')
# timestamp, hour, day, week, month, year, weekday
df['hour'] = df['ts'].dt.hour
df['day'] = df['ts'].dt.day
df['week'] = df['ts'].dt.week
df['month'] = df['ts'].dt.month
df['year'] = df['ts'].dt.year
df['weekday'] = df['ts'].dt.weekday

In [36]:
df.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,ts,userAgent,userId,start_time,hour,day,week,month,year,weekday
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,...,2018-11-30 00:22:07.796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,Manowar,Logged In,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,...,2018-11-30 01:08:41.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73,2018-11-30 01:08:41.796,1,30,48,11,2018,4


In [12]:
time_df = df[['start_time','hour','day','week','month','year','weekday']]
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
time_table_insert = ("INSERT INTO time (start_time, hour, day, week, month, year, weekday)\
                    VALUES (%s, %s, %s, %s, %s, %s, %s)")
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId','firstName','lastName','gender','level']].drop_duplicates()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
user_table_insert = ("INSERT INTO users (user_id, first_name,last_name, gender,level )\
                    VALUES (%s, %s, %s, %s, %s)")
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [32]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId',
       'start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'],
      dtype='object')

In [17]:
song_select = ("SELECT s.song_id, s.artist_id FROM songs s\
                INNER JOIN artists a ON s.artist_id = a.artist_id\
                WHERE s.title = %s\
                AND a.name = %s\
                AND s.duration = %s")
songplay_table_insert = ("INSERT INTO songplays (start_time, user_id, level, song_id,  \
                        artist_id, session_id ,location, user_agent)\
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)")

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    print (row.song, row.artist, row.length)
    
    if results:
        
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.start_time, row.userId, row.level, songid, artistid,row.sessionId, row.location,
                    row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Jim Henson's Dead Stephen Lynch 182.85669
Shell Shock Manowar 247.562
Women Lose Weight (Feat: Slick Rick) Morcheeba 257.41016
Won't Go Home Without You Maroon 5 231.23546
Hey_ Soul Sister Train 216.76363
I'm In Miami Bitch LMFAO 227.99628
Sexy Bitch DJ Dizzy 221.1522
The Cure & The Cause (Dennis Ferrer Remix) Fish Go Deep & Tracey K 377.41669
None None nan
Staring At Me M83 96.1824
The Second You Sleep Saybia 257.25342
Wide Eyes Local Natives 266.05669
La Resistance (Medley) (LP Version) South Park 112.97914
Restless UNKLE Feat. Josh Homme 307.19955
Noc... Justyna Steczkowska 333.53098
Like You Evanescence 256.91383
God Put A Smile Upon Your Face Coldplay 297.35138
Playboy Hot Chip 333.50485
Every Time You Go Ellie Goulding 205.06077
Natural Anthem (Album) Postal Service 307.53914
None None nan
None None nan
None None nan
Taylor Jack Johnson 240.06485
Naked As We Can Iron And Wine 153.05098
Fantasy The xx 158.24934
Epilogue The Antlers 328.88118
Groovin' Fattburger 217.20771
None None

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [37]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.